<a href="https://colab.research.google.com/github/lhcaleo/NeurIPS_Challenge/blob/master/551Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mount Drive**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# **Load File Location**

In [4]:
cd '/content/drive/My Drive/Colab Notebooks/dragonnet-master/src' 

/content/drive/My Drive/Colab Notebooks/dragonnet-master/src


# **idhp_dragonnet.py**

In [5]:
from experiment.models import *
import os
import glob
import argparse
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import keras.backend as K
from keras.optimizers import rmsprop, SGD, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau, TerminateOnNaN
from experiment.idhp_data import *

print(tf.__version__)

def _split_output(yt_hat, t, y, y_scaler, x, index):
    q_t0 = y_scaler.inverse_transform(yt_hat[:, 0].copy())
    q_t1 = y_scaler.inverse_transform(yt_hat[:, 1].copy())
    g = yt_hat[:, 2].copy()

    if yt_hat.shape[1] == 4:
        eps = yt_hat[:, 3][0]
    else:
        eps = np.zeros_like(yt_hat[:, 2])

    y = y_scaler.inverse_transform(y.copy())
    var = "average propensity for treated: {} and untreated: {}".format(g[t.squeeze() == 1.].mean(),
                                                                        g[t.squeeze() == 0.].mean())
    print(var)

    return {'q_t0': q_t0, 'q_t1': q_t1, 'g': g, 't': t, 'y': y, 'x': x, 'index': index, 'eps': eps}


def train_and_predict_dragons(t, y_unscaled, x, targeted_regularization=True, output_dir='',
                              knob_loss=dragonnet_loss_binarycross, ratio=1, dragon='', val_split=0.2, batch_size=64):
    print(val_split)
    verbose = 0
    y_scaler = StandardScaler().fit(y_unscaled)
    y = y_scaler.transform(y_unscaled)
    train_outputs = []
    test_outputs = []

    if dragon == 'tarnet':
        dragonnet = make_tarnet(x.shape[1], 0.01)

    elif dragon == 'dragonnet':
        print("I am here making dragonnet")
        dragonnet = make_dragonnet(x.shape[1], 0.01)

    metrics = [regression_loss, binary_classification_loss, treatment_accuracy, track_epsilon]

    if targeted_regularization:
        loss = make_tarreg_loss(ratio=ratio, dragonnet_loss=knob_loss)
    else:
        loss = knob_loss

    # for reporducing the IHDP experimemt

    i = 0
    tf.random.set_random_seed(i)
    np.random.seed(i)

    test_size = 0.0001
    train_index, test_index = train_test_split(np.arange(x.shape[0]), test_size=test_size)
    if test_size == 0.0001:
        test_index = train_index

    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    t_train, t_test = t[train_index], t[test_index]

    yt_train = np.concatenate([y_train, t_train], 1)

    import time;
    start_time = time.time()

    dragonnet.compile(
        optimizer=Adam(lr=1e-3),
        loss=loss, metrics=metrics)

    adam_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=2, min_delta=0.),
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=1e-8, cooldown=0, min_lr=0)

    ]

    dragonnet.fit(x_train, yt_train, callbacks=adam_callbacks,
                  validation_split=val_split,
                  epochs=100,
                  batch_size=batch_size, verbose=verbose)

    sgd_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=40, min_delta=0.),
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=0., cooldown=0, min_lr=0)
    ]

    sgd_lr = 1e-5
    momentum = 0.9
    dragonnet.compile(optimizer=SGD(lr=sgd_lr, momentum=momentum, nesterov=True), loss=loss,
                      metrics=metrics)
    dragonnet.fit(x_train, yt_train, callbacks=sgd_callbacks,
                  validation_split=val_split,
                  epochs=300,
                  batch_size=batch_size, verbose=verbose)

    elapsed_time = time.time() - start_time
    print("***************************** elapsed_time is: ", elapsed_time)

    yt_hat_test = dragonnet.predict(x_test)
    yt_hat_train = dragonnet.predict(x_train)

    test_outputs += [_split_output(yt_hat_test, t_test, y_test, y_scaler, x_test, test_index)]
    train_outputs += [_split_output(yt_hat_train, t_train, y_train, y_scaler, x_train, train_index)]
    K.clear_session()

    return test_outputs, train_outputs


def train_and_predict_ned(t, y_unscaled, x, targeted_regularization=True, output_dir='',
                          knob_loss=dragonnet_loss_binarycross, ratio=1., dragon='', val_split=0.2, batch_size=64):
    verbose = 0
    y_scaler = StandardScaler().fit(y_unscaled)
    y = y_scaler.transform(y_unscaled)

    train_outputs = []
    test_outputs = []

    nednet = make_ned(x.shape[1], 0.01)

    metrics_ned = [ned_loss]
    metrics_cut = [regression_loss]

    # for reproducing the ihdp result
    i = 0

    tf.random.set_random_seed(i)
    np.random.seed(i)

    # change the test_size to get in sample and out sample estimates

    test_size = 0.0001
    train_index, test_index = train_test_split(np.arange(x.shape[0]), test_size=test_size)
    if test_size == 0.0001:
        test_index = train_index

    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    t_train, t_test = t[train_index], t[test_index]
    yt_train = np.concatenate([y_train, t_train], 1)


    nednet.compile(
        optimizer=Adam(lr=1e-3),
        loss=ned_loss, metrics=metrics_ned)

    adam_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=2, min_delta=0.),
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=1e-8, cooldown=0, min_lr=0)
    ]

    nednet.fit(x_train, yt_train, callbacks=adam_callbacks,
               validation_split=val_split,
               epochs=100,
               batch_size=batch_size, verbose=verbose)

    sgd_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=40, min_delta=0.),
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=0., cooldown=0, min_lr=0)]

    sgd_lr = 1e-5
    momentum = 0.9
    nednet.compile(optimizer=SGD(lr=sgd_lr, momentum=momentum, nesterov=True), loss=ned_loss,
                   metrics=metrics_ned)
    print(nednet.summary())
    nednet.fit(x_train, yt_train, callbacks=sgd_callbacks,
               validation_split=val_split,
               epochs=300,
               batch_size=batch_size, verbose=verbose)

    t_hat_test = nednet.predict(x_test)[:, 1]
    t_hat_train = nednet.predict(x_train)[:, 1]

    # cutting the activation layer
    cut_net = post_cut(nednet, x.shape[1], 0.01)

    cut_net.compile(
        optimizer=Adam(lr=1e-3),
        loss=dead_loss, metrics=metrics_cut)

    adam_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=2, min_delta=0.),
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=1e-8, cooldown=0, min_lr=0)
    ]

    cut_net.fit(x_train, yt_train, callbacks=adam_callbacks,
                validation_split=val_split,
                epochs=100,
                batch_size=batch_size, verbose=verbose)

    sgd_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=40, min_delta=0.),
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=0., cooldown=0, min_lr=0)]

    sgd_lr = 1e-5
    momentum = 0.9
    cut_net.compile(optimizer=SGD(lr=sgd_lr, momentum=momentum, nesterov=True), loss=dead_loss,
                    metrics=metrics_cut)

    cut_net.fit(x_train, yt_train, callbacks=sgd_callbacks,
                validation_split=val_split,
                epochs=300,
                batch_size=batch_size, verbose=verbose)

    y_hat_test = cut_net.predict(x_test)
    y_hat_train = cut_net.predict(x_train)

    yt_hat_test = np.concatenate([y_hat_test, t_hat_test.reshape(-1, 1)], 1)
    yt_hat_train = np.concatenate([y_hat_train, t_hat_train.reshape(-1, 1)], 1)

    test_outputs += [_split_output(yt_hat_test, t_test, y_test, y_scaler, x_test, test_index)]
    train_outputs += [_split_output(yt_hat_train, t_train, y_train, y_scaler, x_train, train_index)]
    K.clear_session()

    return test_outputs, train_outputs


def run_ihdp(data_base_dir='/Users/claudiashi/data/ihdp_csv', output_dir='~/result/ihdp/',
             knob_loss=dragonnet_loss_binarycross,
             ratio=1., dragon=''):
    print("the dragon is {}".format(dragon))

    simulation_files = sorted(glob.glob("{}/*.csv".format(data_base_dir)))

    for idx, simulation_file in enumerate(simulation_files):

        simulation_output_dir = os.path.join(output_dir, str(idx))

        os.makedirs(simulation_output_dir, exist_ok=True)

        x = load_and_format_covariates_ihdp(simulation_file)
        t, y, y_cf, mu_0, mu_1 = load_all_other_crap(simulation_file)
        np.savez_compressed(os.path.join(simulation_output_dir, "simulation_outputs.npz"),
                            t=t, y=y, y_cf=y_cf, mu_0=mu_0, mu_1=mu_1)

        for is_targeted_regularization in [True, False]:
            print("Is targeted regularization: {}".format(is_targeted_regularization))
            if dragon == 'nednet':
                test_outputs, train_output = train_and_predict_ned(t, y, x,
                                                                   targeted_regularization=is_targeted_regularization,
                                                                   output_dir=simulation_output_dir,
                                                                   knob_loss=knob_loss, ratio=ratio, dragon=dragon,
                                                                   val_split=0.3, batch_size=64)
            else:

                test_outputs, train_output = train_and_predict_dragons(t, y, x,
                                                                       targeted_regularization=is_targeted_regularization,
                                                                       output_dir=simulation_output_dir,
                                                                       knob_loss=knob_loss, ratio=ratio, dragon=dragon,
                                                                       val_split=0.2, batch_size=64)

            if is_targeted_regularization:
                train_output_dir = os.path.join(simulation_output_dir, "targeted_regularization")
            else:
                train_output_dir = os.path.join(simulation_output_dir, "baseline")
            os.makedirs(train_output_dir, exist_ok=True)

            # save the outputs of for each split (1 per npz file)
            for num, output in enumerate(test_outputs):
                np.savez_compressed(os.path.join(train_output_dir, "{}_replication_test.npz".format(num)),
                                    **output)

            for num, output in enumerate(train_output):
                np.savez_compressed(os.path.join(train_output_dir, "{}_replication_train.npz".format(num)),
                                    **output)


def turn_knob(data_base_dir='/Users/claudiashi/data/test/', knob='dragonnet',
              output_base_dir=''):
    output_dir = os.path.join(output_base_dir, knob)

    if knob == 'dragonnet':
        run_ihdp(data_base_dir=data_base_dir, output_dir=output_dir, dragon='dragonnet')

    if knob == 'tarnet':
        run_ihdp(data_base_dir=data_base_dir, output_dir=output_dir, dragon='tarnet')

    if knob == 'nednet':
        run_ihdp(data_base_dir=data_base_dir, output_dir=output_dir, dragon='nednet')




Using TensorFlow backend.


1.15.0


# **Run Dragonnet**

In [6]:

def main():
    #turn_knob(args.data_base_dir, args.knob, args.folder, args.output_base_dir, args.dataset)
    turn_knob(data_base_dir='/content/drive/My Drive/Colab Notebooks/dragonnet-master/dat/ihdp/csv', 
    knob='dragonnet', 
    output_base_dir='/content/drive/My Drive/Colab Notebooks/dragonnet-master/src/Result') 


if __name__ == '__main__':
    main()

print("run_dragonnet_done")

the dragon is dragonnet
Is targeted regularization: True
0.2
I am here making dragonnet






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where








***************************** elapsed_time is:  11.101672887802124
average propensity for treated: 0.26416459679603577 and untreated: 0.17697075009346008
average propensity for treated: 0.26416459679603577 and untreated: 0.17697075009346008


Is targeted regularization: False
0.2
I am here making dragonnet
***************************** elapsed_time is:  5.841557502746582
average propensity for treated: 0.27276259660720825 and untreated: 0.17516310513019562
average propensity for treated: 0.27276259660720825 and untreated: 0.17516310513019562
Is targeted regularization: True
0.2
I am here making dragonnet
***************************** elapsed_time is:  12.943016529083252
average propensity for treated: 0.2528091371059418 and untreated: 0.17777119576931
average propensity for treated: 0.25280913

# **Run Tarnet**

In [14]:
def main():
    #turn_knob(args.data_base_dir, args.knob, args.folder, args.output_base_dir, args.dataset)
    turn_knob(data_base_dir='/content/drive/My Drive/Colab Notebooks/dragonnet-master/dat/ihdp/csv',  # need to change 
    knob='tarnet', 
    output_base_dir='/content/drive/My Drive/Colab Notebooks/dragonnet-master/src/Result') # need to change 


if __name__ == '__main__':
    main()

print("run_tarnet_done")

the dragon is tarnet
Is targeted regularization: True
0.2
***************************** elapsed_time is:  10.75789499282837
average propensity for treated: 0.22903676331043243 and untreated: 0.19930854439735413
average propensity for treated: 0.22903676331043243 and untreated: 0.19930854439735413
Is targeted regularization: False
0.2
***************************** elapsed_time is:  14.935969591140747
average propensity for treated: 0.22028951346874237 and untreated: 0.19980835914611816
average propensity for treated: 0.22028951346874237 and untreated: 0.19980835914611816
Is targeted regularization: True
0.2
***************************** elapsed_time is:  14.626362562179565
average propensity for treated: 0.23393040895462036 and untreated: 0.20653831958770752
average propensity for treated: 0.23393040895462036 and untreated: 0.20653831958770752
Is targeted regularization: False
0.2
***************************** elapsed_time is:  14.553131580352783
average propensity for treated: 0.227316

# **Run Nednet**

In [15]:
def main():
    #turn_knob(args.data_base_dir, args.knob, args.folder, args.output_base_dir, args.dataset)
    turn_knob(data_base_dir='/content/drive/My Drive/Colab Notebooks/dragonnet-master/dat/ihdp/csv',  # need to change 
    knob='nednet', 
    output_base_dir='/content/drive/My Drive/Colab Notebooks/dragonnet-master/src/Result') # need to change 


if __name__ == '__main__':
    main()

print("run_nednet_done")

the dragon is nednet
Is targeted regularization: True
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 25)           0                                            
__________________________________________________________________________________________________
ned_hidden1 (Dense)             (None, 200)          5200        input[0][0]                      
__________________________________________________________________________________________________
ned_hidden2 (Dense)             (None, 200)          40200       ned_hidden1[0][0]                
__________________________________________________________________________________________________
ned_hidden3 (Dense)             (None, 200)          40200       ned_hidden2[0][0]                
______________________________________

# **idhp_ate.py**
\delta_in

In [0]:
import copy

from numpy import load

from semi_parametric_estimation.ate import *


def load_truth(replication, knob):
    """
    loading ground truth data
    """

    file_path = '/content/drive/My Drive/Colab Notebooks/dragonnet-master/src/Result/{}/{}/simulation_outputs.npz'.format(knob, replication)
    data = load(file_path)
    mu_0 = data['mu_0']
    mu_1 = data['mu_1']

    return mu_1, mu_0


def load_data(knob='default', replication=1, model='baseline', train_test='test'):
    """
    loading train test experiment results
    """

    file_path = '/content/drive/My Drive/Colab Notebooks/dragonnet-master/src/Result/{}/'.format(knob)
    data = load(file_path + '{}/{}/0_replication_{}.npz'.format(replication, model, train_test))

    return data['q_t0'].reshape(-1, 1), data['q_t1'].reshape(-1, 1), data['g'].reshape(-1, 1), \
           data['t'].reshape(-1, 1), data['y'].reshape(-1, 1), data['index'].reshape(-1, 1), data['eps'].reshape(-1, 1)


def get_estimate(q_t0, q_t1, g, t, y_dragon, index, eps, truncate_level=0.01):
    """
    getting the back door adjustment & TMLE estimation
    """

    psi_n = psi_naive(q_t0, q_t1, g, t, y_dragon, truncate_level=truncate_level)
    psi_tmle, psi_tmle_std, eps_hat, initial_loss, final_loss, g_loss = psi_tmle_cont_outcome(q_t0, q_t1, g, t,
                                                                                              y_dragon,
                                                                                              truncate_level=truncate_level)
    return psi_n, psi_tmle, initial_loss, final_loss, g_loss


def make_table(train_test='train', n_replication=50):
    dict = {'tarnet': {'baseline': {'back_door': 0, }, 'targeted_regularization': 0},
            'dragonnet': {'baseline': 0, 'targeted_regularization': 0},
            'nednet': {'baseline': 0, 'targeted_regularization': 0}}
    tmle_dict = copy.deepcopy(dict)

    for knob in ['dragonnet','tarnet','nednet']:
        for model in ['baseline', 'targeted_regularization']:
            simple_errors, tmle_errors = [], []
            for rep in range(n_replication):
                q_t0, q_t1, g, t, y_dragon, index, eps = load_data(knob, rep, model, train_test)
                a, b = load_truth(rep, knob)
                mu_1, mu_0 = a[index], b[index]

                truth = (mu_1 - mu_0).mean()

                psi_n, psi_tmle, initial_loss, final_loss, g_loss = get_estimate(q_t0, q_t1, g, t, y_dragon, index, eps,
                                                                                 truncate_level=0.01)

                err = abs(truth - psi_n).mean()
                tmle_err = abs(truth - psi_tmle).mean()
                simple_errors.append(err)
                tmle_errors.append(tmle_err)

            dict[knob][model] = np.mean(simple_errors)
            tmle_dict[knob][model] = np.mean(tmle_errors)

    return dict, tmle_dict





# **idhp_ate_main.py**

In [16]:
def main():
    dict, tmle_dict = make_table()
    print("The back door adjustment result is below")
    print(dict)

    print("the tmle estimator result is this ")
    print(tmle_dict)


if __name__ == "__main__":
    main()

The back door adjustment result is below
{'tarnet': {'baseline': 0.12504421744050617, 'targeted_regularization': 0.1191920418706454}, 'dragonnet': {'baseline': 0.12754051156209403, 'targeted_regularization': 0.14132166487134246}, 'nednet': {'baseline': 0.18165883789319032, 'targeted_regularization': 0.18931748731470793}}
the tmle estimator result is this 
{'tarnet': {'baseline': 0.12316339778468582, 'targeted_regularization': 0.11989218527556239}, 'dragonnet': {'baseline': 0.1224153766497898, 'targeted_regularization': 0.1246210706505555}, 'nednet': {'baseline': 0.13364663677613112, 'targeted_regularization': 0.14120976249201458}}
